In [36]:
import torch
from torch import nn
import os
import pandas as pd
from torchvision.io import read_image
from sklearn.model_selection import train_test_split

# Creating the Dataset

In [37]:
data = pd.read_csv("./data/sequence_descriptors_labels.csv")
data = data.iloc[:, 5:]
data.head()


,BLOSUM1,BLOSUM2,BLOSUM3,BLOSUM4,BLOSUM5,BLOSUM6,BLOSUM7,BLOSUM8,BLOSUM9,BLOSUM10,...,VHSE5,VHSE6,VHSE7,VHSE8,Z1,Z2,Z3,Z4,Z5,label
0,0.350,-0.392,-0.020,-0.246,0.146,0.372,0.432,0.256,0.302,-0.184,...,0.614,-0.472,1.094,-0.338,0.596,-1.266,-1.146,0.236,0.360,0
1,-0.170,-0.610,-0.012,0.000,-0.138,0.408,0.364,0.270,0.222,0.028,...,0.346,-0.928,0.736,-0.390,-0.640,-1.790,-0.990,-0.230,0.350,0
2,0.138,-0.568,0.246,-0.076,0.030,0.390,0.522,0.220,0.538,0.052,...,0.290,-0.788,0.840,-0.460,0.186,-1.908,-0.528,-0.090,0.558,0
3,0.350,-0.392,-0.020,-0.246,0.146,0.372,0.432,0.256,0.302,-0.184,...,0.614,-0.472,1.094,-0.338,0.596,-1.266,-1.146,0.236,0.360,0
4,-0.112,-0.652,-0.518,-0.188,0.136,0.190,0.164,0.122,0.256,-0.190,...,0.780,-0.558,0.660,-0.096,-0.592,-0.800,-1.560,0.232,0.488,0


In [38]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class CSVDataset(Dataset):
    def __init__(self, data, target_column):
        self.data = data
        self.features = self.data.drop(columns=[target_column]).values  # Input features
        self.labels = self.data[target_column].values  # Target values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.features[idx], dtype=torch.float32)
        y = torch.tensor(self.labels[idx], dtype=torch.float32)
        return x, y  # Returns a tuple (features, label)

# Example Usage
dataset = CSVDataset(data, target_column="label")  # Replace with your CSV file
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Split data
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [39]:
# device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cpu device


In [40]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(68, 36),
            nn.ReLU(),
            nn.Linear(36, 36),
            nn.ReLU(),
            nn.Linear(36, 3),
            nn.Softmax(),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [41]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=68, out_features=36, bias=True)
    (1): ReLU()
    (2): Linear(in_features=36, out_features=36, bias=True)
    (3): ReLU()
    (4): Linear(in_features=36, out_features=3, bias=True)
    (5): Softmax(dim=None)
  )
)


## Random Test

In [42]:
X = torch.rand(1, 68, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1])


/Volumes/TH External/Uni (Honours)/_Class Work/Honours Science 1/pytorch-mnist/.env/lib/python3.13/site-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [43]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
batch_size = 64
epochs = 5

In [48]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y.long())

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y.long()).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [49]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.082701  [   32/26139]
loss: 0.895220  [ 6432/26139]
loss: 1.082695  [12832/26139]
loss: 1.176481  [19232/26139]
loss: 0.895161  [25632/26139]


/Volumes/TH External/Uni (Honours)/_Class Work/Honours Science 1/pytorch-mnist/.env/lib/python3.13/site-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


loss: 0.957660  [32032/26139]
loss: 0.863950  [38432/26139]
loss: 1.145199  [44832/26139]
loss: 0.863958  [51232/26139]
Test Error: 
 Accuracy: 55.1%, Avg loss: 1.001269 

Epoch 2
-------------------------------
loss: 1.082702  [   32/26139]
loss: 1.020191  [ 6432/26139]
loss: 0.926443  [12832/26139]
loss: 0.957697  [19232/26139]
loss: 1.145201  [25632/26139]
loss: 0.957696  [32032/26139]
loss: 1.020194  [38432/26139]
loss: 1.020188  [44832/26139]
loss: 1.020187  [51232/26139]
Test Error: 
 Accuracy: 55.1%, Avg loss: 1.001269 

Epoch 3
-------------------------------
loss: 1.020236  [   32/26139]
loss: 0.926447  [ 6432/26139]
loss: 0.926445  [12832/26139]
loss: 1.051381  [19232/26139]
loss: 1.051445  [25632/26139]
loss: 0.863944  [32032/26139]
loss: 1.020197  [38432/26139]
loss: 1.082652  [44832/26139]
loss: 1.020159  [51232/26139]
Test Error: 
 Accuracy: 55.1%, Avg loss: 1.001269 

Epoch 4
-------------------------------
loss: 0.895241  [   32/26139]
loss: 1.020195  [ 6432/26139]
loss